# Data Preparation
This notebook is used prepare the data for training. Invalid images are removed and labels are created from filenames. The labels are also summarized because some were redundant. <br>
After training the first models, some classes were removed due to poor performance.

In [1]:
import os
import pandas as pd
from PIL import Image
import glob
from sklearn.utils import shuffle
import numpy as np
from PIL import ImageFile

### remove invalid images

In [2]:
for file in glob.glob('/home/dschulte/Medienverarbeitung/*/*'):
    try: 
        Image.open(file)
        
    except:
        print('file: ' + file + ' was not valid.')
        os.remove(file)

In [3]:
for file in glob.glob('/home/jpawick/Medienverarbeitung/augmented_images/*'):
    try: 
        Image.open(file)
        
    except:
        print('file: ' + file + ' was not valid.')
        os.remove(file)

### create labels from filenames for not augmented images

In [4]:
df_data = pd.DataFrame()
df_data_augemented = pd.DataFrame()

In [5]:
list_paths = ['/home/jpawick/Medienverarbeitung/done/*/*', 
              '/home/swehrhahn/Medienverarbeitung/*/*', 
              '/home/dschulte/Medienverarbeitung/*/*']
path_augmentated = '/home/jpawick/Medienverarbeitung/augmented_images/*'

In [6]:
list_file_names = []
list_label_species = []
list_label_diseases = []

for path in list_paths:
#    print(path)
    for path_img in glob.glob(path):
        list_file_names.append(path_img)
        list_label_diseases.append(path_img.replace('/done', '').replace(' ', '_').split('/Medienverarbeitung/')[1].split('/')[0].replace('_done', ''))
        list_label_species.append(path_img.replace('/done', '').split('/Medienverarbeitung/')[1].split('/')[0].replace('_', ' ').split()[0])
        
df_data['filename'] = list_file_names
df_data['label_species'] = list_label_species
df_data['label_disease'] = list_label_diseases

In [7]:
df_data.head()

,filename,label_species,label_disease
0,/home/jpawick/Medienverarbeitung/done/apple_sc...,apple,apple_scab_leaf
1,/home/jpawick/Medienverarbeitung/done/apple_sc...,apple,apple_scab_leaf
2,/home/jpawick/Medienverarbeitung/done/apple_sc...,apple,apple_scab_leaf
3,/home/jpawick/Medienverarbeitung/done/apple_sc...,apple,apple_scab_leaf
4,/home/jpawick/Medienverarbeitung/done/apple_sc...,apple,apple_scab_leaf


### create labels from filenames for augmented images

In [9]:
list_file_names = []
list_label_species = []
list_label_diseases = []

to_replace_prefixes = ['rotated_1', 'rotated_2', 'flip_h_', 'flip_v_']

for path_img in glob.glob(path_augmentated):
    list_file_names.append(path_img)
    
    label_species = path_img 
    for to_replace in to_replace_prefixes:
        label_species = label_species.replace(to_replace, '')
    list_label_species.append(label_species.split('augmented_images/')[1].replace(' ', '_').split('_')[0])
    
    label_disease = label_species.split('augmented_images/')[1].split('_Image')[0].replace(' ', '_').replace('rotated_2', '')
    list_label_diseases.append(label_disease) 
 
df_data_augemented['filename'] = list_file_names
df_data_augemented['label_species'] = list_label_species
df_data_augemented['label_disease'] = list_label_diseases 

### summarize labels to reduce redundancy 

In [12]:
# summarize labels for diseases
list_label_diseases = df_data['label_disease']
list_label_diseases_summarized = []

for label in list_label_diseases:
    if 'rust' in label:
        list_label_diseases_summarized.append('rust_leaf')
    elif 'blight' in label: 
        list_label_diseases_summarized.append('leaf_blight')
    elif label in ['strawberry_leaf', 'apple_leaf', 'tomato_leaf', 'soybean_leaf', 'potato_leaf', 'grape_leaf', 'peach_leaf', 
                   'squash_leaf', 'raspberry_leaf']:
        list_label_diseases_summarized.append('healthy')    
    else: 
        list_label_diseases_summarized.append(label)
df_data['label_diseases_summarized'] = list_label_diseases_summarized

list_label_diseases_augemented = df_data_augemented['label_disease']
list_label_diseases_augemented_summarized = []

for label in list_label_diseases_augemented:
    if 'rust' in label:
        list_label_diseases_augemented_summarized.append('rust_leaf')
    elif 'blight' in label: 
        list_label_diseases_augemented_summarized.append('leaf_blight')
    elif label in ['strawberry_leaf', 'apple_leaf', 'tomato_leaf', 'soybean_leaf', 'potato_leaf', 'grape_leaf', 'peach_leaf', 
                   'squash_leaf', 'raspberry_leaf']:
        list_label_diseases_augemented_summarized.append('healthy')    
    else: 
        list_label_diseases_augemented_summarized.append(label) 
df_data_augemented['label_diseases_summarized'] = list_label_diseases_augemented_summarized

### remove classes that performed bad

In [13]:
# remove some species
df_data = df_data[~df_data.label_species.isin(['bell', 'strawberry', 'raspberry'])]
df_data_augemented = df_data_augemented[~df_data_augemented.label_species.isin(['bell', 'strawberry', 'raspberry'])]

#remove diseases 
df_data = df_data[~df_data.label_diseases_summarized.isin([])]
df_data_augemented = df_data_augemented[~df_data_augemented.label_diseases_summarized.isin([])]

### distribution of the labels

In [14]:
df_data_augemented['label_species'].value_counts()

tomato     4206
soybean    3327
apple      1351
peach      1312
potato      836
corn        758
squash      535
grape       475
Name: label_species, dtype: int64

In [15]:
df_data_augemented['label_diseases_summarized'].value_counts()

rust_leaf                     2896
healthy                       2865
leaf_blight                   2421
tomato_leaf_spot              1301
peach_leaf_bacterial_spot      675
tomato_mold_leaf               600
squash_powdery_mildew_leaf     505
peach_leaf_curl                412
tomato_leaf_mosaic_virus       355
apple_scab_leaf                305
tomato_leaf_yellow_virus       260
grape_leaf_black_rot           205
Name: label_diseases_summerized, dtype: int64

In [16]:
df_data.shape

(2657, 4)

In [17]:
df_data_augemented.shape

(12800, 4)

In [18]:
df_data = shuffle(df_data)
df_data_augemented = shuffle(df_data_augemented)

In [19]:
df_data.to_csv('train_data.csv', index=False)
df_data_augemented.to_csv('train_data_augmented.csv', index=False)